# US Immigration And Its Contributing Factors
### Data Engineering Capstone Project

#### Project Summary
This project studies the relationship between the immigrants to US states and the population demographics, the temperature, and the airports.  
It uses STAR schema to create different dimension and fact tables based on the data sourced publicily.  
The processes developed in this Jupyter Notebook is based on premise tools.  
It uses sqlite to build a data warehouse and develops ETL scripts with Python and SQL. 

The project follows the following steps:
* [Step 1: Scope the Project and Gather Data](#step1)
* [Step 2: Explore and Assess the Data](#step2)
* [Step 3: Define the Data Model](#model)
* [Step 4: Run ETL to Model the Data](#check)
* [Step 5: Complete Project Write Up](#complete)

In [1]:
# !pip install --upgrade pandas

In [2]:
# !pip install --upgrade numpy

In [3]:
# !pip install --upgrade numexpr

In [4]:
# !pip install ipython-sql

In [5]:
# !d:\anaconda3\python.exe -m pip install --upgrade pip

In [6]:
# Do all imports and installs here
import pandas as pd
import numpy as np
import sqlite3
from sqlalchemy import create_engine

In [7]:
# show all the columns
pd.set_option('display.max_columns', None)

In [8]:
# Run sql query
%load_ext sql

<a id='step1'></a>

### Step 1: Scope the Project and Gather Data

#### Scope 
1. **Purpose**: The relationship among immigrants, temperature, population, and airports.
2. **Data**: immigrants, temperature, population, and airports data.
3. **Solution**: dimension tables for temperature, population, airports and fact tables for tourists.
4. **Tools**: sqlite, Python, SQL
#### Describe and Gather Data 
- **Immigrants**: immigrants and their demographics; [Source](https://www.trade.gov/national-travel-and-tourism-office)
- **Temperature**: average temperature; [Source](https://www.kaggle.com/datasets/berkeleyearth/climate-change-earth-surface-temperature-data?resource=download)
- **Population**: population and its demographics; [Source](https://public.opendatasoft.com/explore/dataset/us-cities-demographics/export/)
- **Airports**: airport type and features; [Source](https://datahub.io/core/airport-codes#pandas)

<a id="step2"></a>

### Step 2: Explore and Assess the Data
#### Explore the Data 
[Identify data quality issues, like missing values, duplicate data, etc.](#identify)
- [airport](#airport)
- [immigration](#immigration)
- [cities](#city)
- [temperature](#temperature)
#### Cleaning Steps
[Document steps necessary to clean the data](#clean)

<a id="identify"></a>

#### 2.1 Explore the data ####

<a id="airport"></a>

[airport](https://datahub.io/core/airport-codes#pandas)

In [9]:
# Read data
# read data as "string" format to ensure all the information is correct
airport = pd.read_csv("../airport-codes_csv.csv", dtype="str")

In [10]:
# airport summary
airport.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55075 entries, 0 to 55074
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   ident         55075 non-null  object
 1   type          55075 non-null  object
 2   name          55075 non-null  object
 3   elevation_ft  48069 non-null  object
 4   continent     27356 non-null  object
 5   iso_country   54828 non-null  object
 6   iso_region    55075 non-null  object
 7   municipality  49399 non-null  object
 8   gps_code      41030 non-null  object
 9   iata_code     9189 non-null   object
 10  local_code    28686 non-null  object
 11  coordinates   55075 non-null  object
dtypes: object(12)
memory usage: 5.0+ MB


**Notes**:
1. All types are "object". Need to be changed to the right formats.

In [11]:
# top 5
airport.head()

,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates
0,00A,heliport,Total Rf Heliport,11,NaN,US,US-PA,Bensalem,00A,NaN,00A,"-74.93360137939453, 40.07080078125"
1,00AA,small_airport,Aero B Ranch Airport,3435,NaN,US,US-KS,Leoti,00AA,NaN,00AA,"-101.473911, 38.704022"
2,00AK,small_airport,Lowell Field,450,NaN,US,US-AK,Anchor Point,00AK,NaN,00AK,"-151.695999146, 59.94919968"
3,00AL,small_airport,Epps Airpark,820,NaN,US,US-AL,Harvest,00AL,NaN,00AL,"-86.77030181884766, 34.86479949951172"
4,00AR,closed,Newport Hospital & Clinic Heliport,237,NaN,US,US-AR,Newport,NaN,NaN,NaN,"-91.254898, 35.6087"


**Notes**:
1. "iso_country" contains country and "iso_region" is combined by country and state.
2. "coordinates" needs to be separated by two columns "longitude" and "latitude".

In [12]:
# airport unique values 
airport.nunique()

ident           55075
type                7
name            52144
elevation_ft     5449
continent           6
iso_country       243
iso_region       2810
municipality    27133
gps_code        40850
iata_code        9042
local_code      27436
coordinates     54874
dtype: int64

In [13]:
# NaN rows
airport[airport.isnull().any(axis=1)]

,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates
0,00A,heliport,Total Rf Heliport,11,NaN,US,US-PA,Bensalem,00A,NaN,00A,"-74.93360137939453, 40.07080078125"
1,00AA,small_airport,Aero B Ranch Airport,3435,NaN,US,US-KS,Leoti,00AA,NaN,00AA,"-101.473911, 38.704022"
2,00AK,small_airport,Lowell Field,450,NaN,US,US-AK,Anchor Point,00AK,NaN,00AK,"-151.695999146, 59.94919968"
3,00AL,small_airport,Epps Airpark,820,NaN,US,US-AL,Harvest,00AL,NaN,00AL,"-86.77030181884766, 34.86479949951172"
4,00AR,closed,Newport Hospital & Clinic Heliport,237,NaN,US,US-AR,Newport,NaN,NaN,NaN,"-91.254898, 35.6087"
...,...,...,...,...,...,...,...,...,...,...,...,...
55070,ZYYK,medium_airport,Yingkou Lanqi Airport,0,AS,CN,CN-21,Yingkou,ZYYK,YKH,NaN,"122.3586, 40.542524"
55071,ZYYY,medium_airport,Shenyang Dongta Airport,NaN,AS,CN,CN-21,Shenyang,ZYYY,NaN,NaN,"123.49600219726562, 41.784400939941406"
55072,ZZ-0001,heliport,Sealand Helipad,40,EU,GB,GB-ENG,Sealand,NaN,NaN,NaN,"1.4825, 51.894444"
55073,ZZ-0002,small_airport,Glorioso Islands Airstrip,11,AF,TF,TF-U-A,Grande Glorieuse,NaN,NaN,NaN,"47.296388888900005, -11.584277777799999"


**Notes**:
1. 54397 rows out of 55075 contain NaN.

In [14]:
# NaN columns
airport.columns[airport.isna().any()]

Index(['elevation_ft', 'continent', 'iso_country', 'municipality', 'gps_code',
       'iata_code', 'local_code'],
      dtype='object')

**Notes**:
1. Some columns contain NaN values.

In [15]:
# mo duplicates
len(airport) == len(airport.drop_duplicates())

True

**Notes**:
1. There are no duplicates. 

<a id="immigration"></a>

[Immigration](https://www.trade.gov/national-travel-and-tourism-office)

In [16]:
# i94 data
# read data as "string" format to ensure all the information is correct and drop "Unnamed: 0" column
immigration = pd.read_csv("../immigration_data_sample.csv", dtype="str").drop("Unnamed: 0", axis = 1)

In [17]:
# top 5
immigration.head()

,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,i94bir,i94visa,count,dtadfile,visapost,occup,entdepa,entdepd,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,4084316.0,2016.0,4.0,209.0,209.0,HHW,20566.0,1.0,HI,20573.0,61.0,2.0,1.0,20160422,NaN,NaN,G,O,NaN,M,1955.0,07202016,F,NaN,JL,56582674633.0,00782,WT
1,4422636.0,2016.0,4.0,582.0,582.0,MCA,20567.0,1.0,TX,20568.0,26.0,2.0,1.0,20160423,MTR,NaN,G,R,NaN,M,1990.0,10222016,M,NaN,*GA,94361995930.0,XBLNG,B2
2,1195600.0,2016.0,4.0,148.0,112.0,OGG,20551.0,1.0,FL,20571.0,76.0,2.0,1.0,20160407,NaN,NaN,G,O,NaN,M,1940.0,07052016,M,NaN,LH,55780468433.0,00464,WT
3,5291768.0,2016.0,4.0,297.0,297.0,LOS,20572.0,1.0,CA,20581.0,25.0,2.0,1.0,20160428,DOH,NaN,G,O,NaN,M,1991.0,10272016,M,NaN,QR,94789696030.0,00739,B2
4,985523.0,2016.0,4.0,111.0,111.0,CHM,20550.0,3.0,NY,20553.0,19.0,2.0,1.0,20160406,NaN,NaN,Z,K,NaN,M,1997.0,07042016,F,NaN,NaN,42322572633.0,LAND,WT


**Notes**:
1. "i94addr" contains "State" which could be a foreign key to link the above airport data.

In [18]:
# data information
immigration.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 28 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   cicid     1000 non-null   object
 1   i94yr     1000 non-null   object
 2   i94mon    1000 non-null   object
 3   i94cit    1000 non-null   object
 4   i94res    1000 non-null   object
 5   i94port   1000 non-null   object
 6   arrdate   1000 non-null   object
 7   i94mode   1000 non-null   object
 8   i94addr   941 non-null    object
 9   depdate   951 non-null    object
 10  i94bir    1000 non-null   object
 11  i94visa   1000 non-null   object
 12  count     1000 non-null   object
 13  dtadfile  1000 non-null   object
 14  visapost  382 non-null    object
 15  occup     4 non-null      object
 16  entdepa   1000 non-null   object
 17  entdepd   954 non-null    object
 18  entdepu   0 non-null      object
 19  matflag   954 non-null    object
 20  biryear   1000 non-null   object
 21  dtaddto   1000 

**Notes**:
1. All types are "object". Need to be changed to the right formats.

In [19]:
# unique values
immigration.nunique()

cicid       1000
i94yr          1
i94mon         1
i94cit        88
i94res        91
i94port       70
arrdate       30
i94mode        4
i94addr       51
depdate      109
i94bir        85
i94visa        3
count          1
dtadfile      39
visapost      97
occup          3
entdepa        9
entdepd       10
entdepu        0
matflag        1
biryear       85
dtaddto       99
gender         3
insnum        29
airline      101
admnum      1000
fltno        502
visatype      10
dtype: int64

In [20]:
# NaN rows
immigration[immigration.isnull().any(axis=1)]

,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,i94bir,i94visa,count,dtadfile,visapost,occup,entdepa,entdepd,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,4084316.0,2016.0,4.0,209.0,209.0,HHW,20566.0,1.0,HI,20573.0,61.0,2.0,1.0,20160422,NaN,NaN,G,O,NaN,M,1955.0,07202016,F,NaN,JL,56582674633.0,00782,WT
1,4422636.0,2016.0,4.0,582.0,582.0,MCA,20567.0,1.0,TX,20568.0,26.0,2.0,1.0,20160423,MTR,NaN,G,R,NaN,M,1990.0,10222016,M,NaN,*GA,94361995930.0,XBLNG,B2
2,1195600.0,2016.0,4.0,148.0,112.0,OGG,20551.0,1.0,FL,20571.0,76.0,2.0,1.0,20160407,NaN,NaN,G,O,NaN,M,1940.0,07052016,M,NaN,LH,55780468433.0,00464,WT
3,5291768.0,2016.0,4.0,297.0,297.0,LOS,20572.0,1.0,CA,20581.0,25.0,2.0,1.0,20160428,DOH,NaN,G,O,NaN,M,1991.0,10272016,M,NaN,QR,94789696030.0,00739,B2
4,985523.0,2016.0,4.0,111.0,111.0,CHM,20550.0,3.0,NY,20553.0,19.0,2.0,1.0,20160406,NaN,NaN,Z,K,NaN,M,1997.0,07042016,F,NaN,NaN,42322572633.0,LAND,WT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,4288772.0,2016.0,4.0,135.0,135.0,LVG,20567.0,1.0,NV,20572.0,32.0,2.0,1.0,20160423,NaN,NaN,G,O,NaN,M,1984.0,07212016,M,NaN,VS,59140652133.0,00043,WT
996,2947585.0,2016.0,4.0,261.0,261.0,PSP,20560.0,1.0,HI,20567.0,35.0,1.0,1.0,20160416,JDD,NaN,G,O,NaN,M,1981.0,10152016,M,NaN,SV,93711864730.0,00041,B1
997,2883298.0,2016.0,4.0,111.0,111.0,MIA,20560.0,1.0,FL,20566.0,39.0,2.0,1.0,20160416,NaN,NaN,G,O,NaN,M,1977.0,07142016,M,NaN,AF,56277467733.0,00090,WT
998,2264857.0,2016.0,4.0,582.0,582.0,ATL,20556.0,1.0,WI,20559.0,35.0,1.0,1.0,20160412,MTR,NaN,G,O,NaN,M,1981.0,10112016,M,NaN,EV,93340353430.0,05510,B1


**Notes**:
1. 1000 rows out of 1000 contain NaN. 

In [21]:
# NaN columns
immigration.columns[immigration.isna().any()]

Index(['i94addr', 'depdate', 'visapost', 'occup', 'entdepd', 'entdepu',
       'matflag', 'gender', 'insnum', 'airline', 'fltno'],
      dtype='object')

**Notes**:
1. Some columns contain NaN values.

In [22]:
# duplicates
len(immigration) == len(immigration.drop_duplicates())

True

**Notes**:
1. There are no duplicates.

<a id="city"></a>

[Cities](https://public.opendatasoft.com/explore/dataset/us-cities-demographics/export/)

In [23]:
# cities
# read data as "string" format to ensure all the information is correct and assign delimiter ";"
cities = pd.read_csv("../us-cities-demographics.csv", delimiter=";", dtype="str")

In [24]:
# top 5
cities.head()

,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,Race,Count
0,Wichita,Kansas,34.6,192354,197601,389955,23978,40270,2.56,KS,American Indian and Alaska Native,8791
1,Allen,Pennsylvania,33.5,60626,59581,120207,5691,19652,2.67,PA,Black or African-American,22304
2,Danbury,Connecticut,37.3,43435,41227,84662,3752,25675,2.74,CT,Black or African-American,8454
3,Nashville,Tennessee,34.1,314231,340365,654596,27942,88193,2.39,TN,Hispanic or Latino,67526
4,Stamford,Connecticut,35.4,64941,63936,128877,2269,44003,2.7,CT,Asian,11013


**Notes**:
1. "State Code" can be used as a foreign key to join the above two tables.
2. Notice that "Race" and "Count" columns need to be transformed from long to wide so each race can be a separate column containing the population for that race.  

In [25]:
# data information
cities.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2891 entries, 0 to 2890
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   City                    2891 non-null   object
 1   State                   2891 non-null   object
 2   Median Age              2891 non-null   object
 3   Male Population         2888 non-null   object
 4   Female Population       2888 non-null   object
 5   Total Population        2891 non-null   object
 6   Number of Veterans      2878 non-null   object
 7   Foreign-born            2878 non-null   object
 8   Average Household Size  2875 non-null   object
 9   State Code              2891 non-null   object
 10  Race                    2891 non-null   object
 11  Count                   2891 non-null   object
dtypes: object(12)
memory usage: 271.2+ KB


**Notes**:
1. All types are "object". Need to be changed to the right formats.

In [26]:
# unique information
cities.nunique()

City                       567
State                       49
Median Age                 180
Male Population            593
Female Population          594
Total Population           594
Number of Veterans         577
Foreign-born               587
Average Household Size     161
State Code                  49
Race                         5
Count                     2785
dtype: int64

In [27]:
# NaN rows
cities[cities.isnull().any(axis=1)]

,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,Race,Count
5,San Juan,Puerto Rico,41.4,155408,186829,342237,NaN,NaN,NaN,PR,Hispanic or Latino,335559
9,Caguas,Puerto Rico,40.4,34743,42265,77008,NaN,NaN,NaN,PR,Hispanic or Latino,76349
280,Bayamón,Puerto Rico,39.4,80128,90131,170259,NaN,NaN,NaN,PR,Hispanic or Latino,169155
740,The Villages,Florida,70.5,NaN,NaN,72590,15231,4034,NaN,FL,White,72211
806,San Juan,Puerto Rico,41.4,155408,186829,342237,NaN,NaN,NaN,PR,American Indian and Alaska Native,4031
908,The Villages,Florida,70.5,NaN,NaN,72590,15231,4034,NaN,FL,Black or African-American,331
1121,Guaynabo,Puerto Rico,42.2,33066,37426,70492,NaN,NaN,NaN,PR,American Indian and Alaska Native,589
1196,Mayagüez,Puerto Rico,38.1,30799,35782,66581,NaN,NaN,NaN,PR,Asian,235
1443,Mayagüez,Puerto Rico,38.1,30799,35782,66581,NaN,NaN,NaN,PR,Hispanic or Latino,65521
1530,Caguas,Puerto Rico,40.4,34743,42265,77008,NaN,NaN,NaN,PR,American Indian and Alaska Native,624


**Notes**:
1. 16 rows out of 2891 contain NaN, which could be dropped.

In [28]:
# NaN columns
cities.columns[cities.isna().any()]

Index(['Male Population', 'Female Population', 'Number of Veterans',
       'Foreign-born', 'Average Household Size'],
      dtype='object')

**Notes**:
1. Some columns contain NaN values.

In [29]:
# duplicates
len(cities) == len(cities.drop_duplicates())

True

**Notes**:
1. There are no duplicates.

<a id="temperature"></a>

[Temperature](https://www.kaggle.com/datasets/berkeleyearth/climate-change-earth-surface-temperature-data?resource=download)

In [30]:
# temperature
# read data as "string" format to ensure all the information is correct
temperature = pd.read_csv("../GlobalLandTemperaturesByState.csv", dtype="str")

In [31]:
# top 5
temperature.head()

,dt,AverageTemperature,AverageTemperatureUncertainty,State,Country
0,1855-05-01,25.544,1.171,Acre,Brazil
1,1855-06-01,24.228,1.103,Acre,Brazil
2,1855-07-01,24.371,1.044,Acre,Brazil
3,1855-08-01,25.427,1.073,Acre,Brazil
4,1855-09-01,25.675,1.014,Acre,Brazil


**Notes**:
1. "State" contain the names of the states in each country, which could work as foreign key to join the above "cities" table and then get the "State Code" to join the other tables.

In [32]:
# data information
temperature.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 645675 entries, 0 to 645674
Data columns (total 5 columns):
 #   Column                         Non-Null Count   Dtype 
---  ------                         --------------   ----- 
 0   dt                             645675 non-null  object
 1   AverageTemperature             620027 non-null  object
 2   AverageTemperatureUncertainty  620027 non-null  object
 3   State                          645675 non-null  object
 4   Country                        645675 non-null  object
dtypes: object(5)
memory usage: 24.6+ MB


**Notes**:
1. All types are "object". Need to be changed to the right formats.

In [33]:
# statistics
temperature.describe()

,dt,AverageTemperature,AverageTemperatureUncertainty,State,Country
count,645675,620027,620027,645675,645675
unique,3239,100924,8040,241,7
top,1990-04-01,18.402,0.255,Vladimir,Russia
freq,241,36,1042,3239,254972


In [34]:
# unique values
temperature.nunique()

dt                                 3239
AverageTemperature               100924
AverageTemperatureUncertainty      8040
State                               241
Country                               7
dtype: int64

In [35]:
# NaN rows
temperature[temperature.isnull().any(axis=1)]

,dt,AverageTemperature,AverageTemperatureUncertainty,State,Country
14,1856-07-01,NaN,NaN,Acre,Brazil
20,1857-01-01,NaN,NaN,Acre,Brazil
21,1857-02-01,NaN,NaN,Acre,Brazil
22,1857-03-01,NaN,NaN,Acre,Brazil
23,1857-04-01,NaN,NaN,Acre,Brazil
...,...,...,...,...,...
641790,1862-10-01,NaN,NaN,Yunnan,China
641791,1862-11-01,NaN,NaN,Yunnan,China
641792,1862-12-01,NaN,NaN,Yunnan,China
643601,2013-09-01,NaN,NaN,Yunnan,China


**Notes**:
1. 25648 rows out of 645675 contain NaN.

In [36]:
# NaN columns
temperature.columns[temperature.isna().any()]

Index(['AverageTemperature', 'AverageTemperatureUncertainty'], dtype='object')

**Notes**:
1. Some columns contain NaN values.

In [37]:
# duplicates
len(temperature) == len(temperature.drop_duplicates())

True

**Notes**:
1. There are no duplicates.

<a id="clean"></a>

#### 2.2 Clean Steps

##### airport
1. to numeric "elevation_ft"
2. Separate "iso_region" to two columns  
3. Separate "coordinates" to two columns and round
4. Drop other airport type only keeping "airport" & not closed
5. Only include US

In [38]:
# US airport 
airportUS = (airport.loc[(airport.iso_country == "US") 
                         & ~(airport.type == "closed") 
                         & airport.type.str.contains("airport")]) 

In [39]:
# change "elevation_ft" to numeric
airportUS["elevation_ft"] = airportUS["elevation_ft"].astype(float)

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [40]:
# separate "iso_region" to "country" and "region"
airportUS[["country", "region"]] = airportUS["iso_region"].str.split("-", 1, expand = True)

D:\Anaconda3\lib\site-packages\pandas\core\frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [41]:
# separate "iso_region" to "country" and "region" and round 4
airportUS[["longitude", "latitude"]] = airportUS["coordinates"].str.split(",", 1, expand = True).astype(float).round(4)

In [42]:
# drop unnecessary columns 
airportUSNew = (airportUS.drop(["iso_country", "iso_region", "coordinates", "country", 
                                "continent","iata_code", "gps_code", "local_code", "municipality"], 
                               axis=1))

In [43]:
# NaN rows
airportUSNew[airportUSNew.isnull().any(axis=1)]

,ident,type,name,elevation_ft,region,longitude,latitude
6875,63CA,small_airport,Desert Air Sky Ranch Airport,NaN,CA,-115.8740,33.4811
6981,65LA,small_airport,Southern Seaplane Airport,NaN,LA,-90.0222,29.8661
7653,6XA4,small_airport,Zadow Airstrip,NaN,TX,-95.9544,29.9917
9006,8AL3,small_airport,Fricks Field Airport,NaN,AL,-86.0886,34.1418
9910,9CL9,small_airport,Spezia Airport,NaN,CA,-121.5340,38.2166
...,...,...,...,...,...,...,...
49947,US-0932,small_airport,CLE,NaN,U-A,0.0000,0.0000
49969,US-0954,small_airport,Dulpicate - Shelter Cove Airport,NaN,CA,124.0711,40.0289
49999,US-0984,small_airport,0c2,NaN,IL,0.0000,0.0000
50032,US-1016,large_airport,JFK,NaN,NY,0.0000,0.0000


In [44]:
# len(airportUSNew) ## 14582 

Only 63 rows out of 14582 rows contain NaN, which could be dropped.

In [45]:
# drop NaN
airportUSNewNoNa = airportUSNew.dropna()

In [46]:
# NaN rows
#airportUSNewNoNa[airportUSNewNoNa.isnull().any(axis=1)]

In [47]:
# duplicates
len(airportUSNewNoNa) == len(airportUSNewNoNa.drop_duplicates())

True

In [48]:
# data information
airportUSNewNoNa.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14519 entries, 1 to 54896
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   ident         14519 non-null  object 
 1   type          14519 non-null  object 
 2   name          14519 non-null  object 
 3   elevation_ft  14519 non-null  float64
 4   region        14519 non-null  object 
 5   longitude     14519 non-null  float64
 6   latitude      14519 non-null  float64
dtypes: float64(3), object(4)
memory usage: 907.4+ KB


In [49]:
# top 5
airportUSNewNoNa.head()

,ident,type,name,elevation_ft,region,longitude,latitude
1,00AA,small_airport,Aero B Ranch Airport,3435.0,KS,-101.4739,38.7040
2,00AK,small_airport,Lowell Field,450.0,AK,-151.6960,59.9492
3,00AL,small_airport,Epps Airpark,820.0,AL,-86.7703,34.8648
5,00AS,small_airport,Fulton Airport,1100.0,OK,-97.8180,34.9428
6,00AZ,small_airport,Cordes Airport,3810.0,AZ,-112.1650,34.3056


##### cities
1. Transform columns "Race" and "Count" from long to wide
2. To numeric: "Male Population", "Female Population", "Total Population", "Number of Veterans", "Foreign-born", "Count"
3. To float: "Median Age", "Average Household Size"

In [50]:
# NaN rows
cities[cities.isnull().any(axis=1)]

,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,Race,Count
5,San Juan,Puerto Rico,41.4,155408,186829,342237,NaN,NaN,NaN,PR,Hispanic or Latino,335559
9,Caguas,Puerto Rico,40.4,34743,42265,77008,NaN,NaN,NaN,PR,Hispanic or Latino,76349
280,Bayamón,Puerto Rico,39.4,80128,90131,170259,NaN,NaN,NaN,PR,Hispanic or Latino,169155
740,The Villages,Florida,70.5,NaN,NaN,72590,15231,4034,NaN,FL,White,72211
806,San Juan,Puerto Rico,41.4,155408,186829,342237,NaN,NaN,NaN,PR,American Indian and Alaska Native,4031
908,The Villages,Florida,70.5,NaN,NaN,72590,15231,4034,NaN,FL,Black or African-American,331
1121,Guaynabo,Puerto Rico,42.2,33066,37426,70492,NaN,NaN,NaN,PR,American Indian and Alaska Native,589
1196,Mayagüez,Puerto Rico,38.1,30799,35782,66581,NaN,NaN,NaN,PR,Asian,235
1443,Mayagüez,Puerto Rico,38.1,30799,35782,66581,NaN,NaN,NaN,PR,Hispanic or Latino,65521
1530,Caguas,Puerto Rico,40.4,34743,42265,77008,NaN,NaN,NaN,PR,American Indian and Alaska Native,624


In [51]:
# len(cities[cities.isnull().any(axis=1)]) ## 16 rows are blank

In [52]:
# len(cities) ## 2891

16 rows out of 2891 are blank. And the status are "Puerto Rico" and "Florida (The Villages)". Thus, dropped these places.

In [53]:
# drop NaN
citiesNoNa = cities.dropna()

In [ ]:
citiesNoNa["Count"] = citiesNoNa["Count"].astype(float)

In [54]:
# transform from long to wide
citiesPop = citiesNoNa[['City', 'State', 'Median Age', 'Male Population',
       'Female Population', 'Total Population', 'Number of Veterans',
       'Foreign-born', 'Average Household Size', 'State Code']]
citiesRace = (pd.pivot_table(citiesNoNa[["City", "Race", "Count"]], 
                             index="City", columns = "Race", values= "Count")
                          .reset_index().reset_index(drop=True).rename_axis(None, axis=1))
citiesNew = citiesPop.merge(citiesRace, on="City", how="left")

In [55]:
# citiesNew.head()

In [56]:
# to integer 
citiesNew[["Male Population", "Female Population", 
           "Total Population", "Number of Veterans", "Foreign-born"]] = (citiesNew[["Male Population", "Female Population", 
                                                        "Total Population", "Number of Veterans", "Foreign-born"
                                                        ]].astype(int))

In [57]:
# to float since some rows contain NaN
# show errors if converting to int
citiesNew[["American Indian and Alaska Native", "Asian", 
           "Black or African-American", "Hispanic or Latino", "White"]] = (citiesNew[["American Indian and Alaska Native", "Asian", 
                                                                                      "Black or African-American", "Hispanic or Latino", "White"]].astype(float))

In [58]:
# to float
citiesNew[["Median Age", "Average Household Size"]] = (citiesNew[["Median Age", "Average Household Size"]].astype(float))

In [59]:
# NaN rows
citiesNew[citiesNew.isnull().any(axis=1)]

,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,American Indian and Alaska Native,Asian,Black or African-American,Hispanic or Latino,White
29,Passaic,New Jersey,30.4,36505,34577,71082,693,24174,3.56,NJ,NaN,3338.0,5230.0,53680.0,30352.0
33,Bayonne,New Jersey,39.7,32705,33598,66303,2225,21899,2.62,NJ,NaN,7171.0,7581.0,19525.0,41431.0
38,Alafaya,Florida,33.5,39504,45760,85264,4176,15842,2.94,FL,NaN,10336.0,6577.0,34897.0,63666.0
79,Miami Beach,Florida,42.5,48090,44221,92311,2265,49908,2.10,FL,NaN,2772.0,4013.0,47446.0,72180.0
83,Sterling Heights,Michigan,39.6,64985,67077,132062,6970,34313,2.66,MI,NaN,12264.0,8054.0,2811.0,112743.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2810,Perris,California,26.9,41623,33336,74959,1321,23277,4.78,CA,NaN,3341.0,9393.0,54398.0,23882.0
2817,Sterling Heights,Michigan,39.6,64985,67077,132062,6970,34313,2.66,MI,NaN,12264.0,8054.0,2811.0,112743.0
2818,Lynwood,California,29.4,35634,36371,72005,776,28061,4.43,CA,NaN,994.0,5346.0,63377.0,48670.0
2868,Sunrise,Florida,37.7,41471,51235,92706,4108,38209,2.98,FL,NaN,3501.0,33951.0,29163.0,54976.0


In [60]:
# len(citiesNew) ## 2875

212 rows out of 2875 contain NaN which are related to "American Indian and Alaska Native". Thus, don't dropped.

In [61]:
# duplicates
len(citiesNew) == len(citiesNew.drop_duplicates())

False

In [62]:
#len(citiesNew.drop_duplicates()) ## 588

In [63]:
# drop duplicates
citiesNewNoDup = citiesNew.drop_duplicates()

In [64]:
# data information
citiesNewNoDup.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 588 entries, 0 to 2195
Data columns (total 15 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   City                               588 non-null    object 
 1   State                              588 non-null    object 
 2   Median Age                         588 non-null    float64
 3   Male Population                    588 non-null    int32  
 4   Female Population                  588 non-null    int32  
 5   Total Population                   588 non-null    int32  
 6   Number of Veterans                 588 non-null    int32  
 7   Foreign-born                       588 non-null    int32  
 8   Average Household Size             588 non-null    float64
 9   State Code                         588 non-null    object 
 10  American Indian and Alaska Native  537 non-null    float64
 11  Asian                              581 non-null    float6

In [65]:
# top 5
citiesNewNoDup.head()

,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,American Indian and Alaska Native,Asian,Black or African-American,Hispanic or Latino,White
0,Wichita,Kansas,34.6,192354,197601,389955,23978,40270,2.56,KS,8791.0,25210.0,5.259200e+04,6.516200e+04,3.059100e+05
1,Allen,Pennsylvania,33.5,60626,59581,120207,5691,19652,2.67,PA,538113.5,133507895.0,1.115207e+09,2.958805e+09,3.709385e+09
2,Danbury,Connecticut,37.3,43435,41227,84662,3752,25675,2.74,CT,1086.0,7350.0,8.454000e+03,2.514500e+04,5.591700e+04
3,Nashville,Tennessee,34.1,314231,340365,654596,27942,88193,2.39,TN,5474.0,27355.0,1.888440e+05,6.752600e+04,4.324470e+05
4,Stamford,Connecticut,35.4,64941,63936,128877,2269,44003,2.70,CT,1416.0,11013.0,2.432900e+04,3.319700e+04,8.562000e+04


##### immigration
1. to date "arrdate", "depdate", "dtadfile"
2. to text "cicid", "i94cit", "i94res", "admnum"
3. to integer "i94yr", "i94mon", "i94mode", "i94bir", "	i94visa", "count"

In [66]:
# NaN rows
immigration[immigration.isnull().any(axis=1)]

,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,i94bir,i94visa,count,dtadfile,visapost,occup,entdepa,entdepd,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,4084316.0,2016.0,4.0,209.0,209.0,HHW,20566.0,1.0,HI,20573.0,61.0,2.0,1.0,20160422,NaN,NaN,G,O,NaN,M,1955.0,07202016,F,NaN,JL,56582674633.0,00782,WT
1,4422636.0,2016.0,4.0,582.0,582.0,MCA,20567.0,1.0,TX,20568.0,26.0,2.0,1.0,20160423,MTR,NaN,G,R,NaN,M,1990.0,10222016,M,NaN,*GA,94361995930.0,XBLNG,B2
2,1195600.0,2016.0,4.0,148.0,112.0,OGG,20551.0,1.0,FL,20571.0,76.0,2.0,1.0,20160407,NaN,NaN,G,O,NaN,M,1940.0,07052016,M,NaN,LH,55780468433.0,00464,WT
3,5291768.0,2016.0,4.0,297.0,297.0,LOS,20572.0,1.0,CA,20581.0,25.0,2.0,1.0,20160428,DOH,NaN,G,O,NaN,M,1991.0,10272016,M,NaN,QR,94789696030.0,00739,B2
4,985523.0,2016.0,4.0,111.0,111.0,CHM,20550.0,3.0,NY,20553.0,19.0,2.0,1.0,20160406,NaN,NaN,Z,K,NaN,M,1997.0,07042016,F,NaN,NaN,42322572633.0,LAND,WT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,4288772.0,2016.0,4.0,135.0,135.0,LVG,20567.0,1.0,NV,20572.0,32.0,2.0,1.0,20160423,NaN,NaN,G,O,NaN,M,1984.0,07212016,M,NaN,VS,59140652133.0,00043,WT
996,2947585.0,2016.0,4.0,261.0,261.0,PSP,20560.0,1.0,HI,20567.0,35.0,1.0,1.0,20160416,JDD,NaN,G,O,NaN,M,1981.0,10152016,M,NaN,SV,93711864730.0,00041,B1
997,2883298.0,2016.0,4.0,111.0,111.0,MIA,20560.0,1.0,FL,20566.0,39.0,2.0,1.0,20160416,NaN,NaN,G,O,NaN,M,1977.0,07142016,M,NaN,AF,56277467733.0,00090,WT
998,2264857.0,2016.0,4.0,582.0,582.0,ATL,20556.0,1.0,WI,20559.0,35.0,1.0,1.0,20160412,MTR,NaN,G,O,NaN,M,1981.0,10112016,M,NaN,EV,93340353430.0,05510,B1


In [67]:
# data information
immigration.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 28 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   cicid     1000 non-null   object
 1   i94yr     1000 non-null   object
 2   i94mon    1000 non-null   object
 3   i94cit    1000 non-null   object
 4   i94res    1000 non-null   object
 5   i94port   1000 non-null   object
 6   arrdate   1000 non-null   object
 7   i94mode   1000 non-null   object
 8   i94addr   941 non-null    object
 9   depdate   951 non-null    object
 10  i94bir    1000 non-null   object
 11  i94visa   1000 non-null   object
 12  count     1000 non-null   object
 13  dtadfile  1000 non-null   object
 14  visapost  382 non-null    object
 15  occup     4 non-null      object
 16  entdepa   1000 non-null   object
 17  entdepd   954 non-null    object
 18  entdepu   0 non-null      object
 19  matflag   954 non-null    object
 20  biryear   1000 non-null   object
 21  dtaddto   1000 

Most of the NaN values are existent in "visapost", "occup", "entdepu", "insnum" columns which are not necessary for data analysis.

In [68]:
# drop unnecessary columns
immigrationNew = immigration.drop(["visapost", "occup", "entdepu", "insnum"], axis=1)

In [69]:
# NaN rows
immigrationNew[immigrationNew.isnull().any(axis=1)]

,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,i94bir,i94visa,count,dtadfile,entdepa,entdepd,matflag,biryear,dtaddto,gender,airline,admnum,fltno,visatype
4,985523.0,2016.0,4.0,111.0,111.0,CHM,20550.0,3.0,NY,20553.0,19.0,2.0,1.0,20160406,Z,K,M,1997.0,07042016,F,NaN,42322572633.0,LAND,WT
8,5227851.0,2016.0,4.0,131.0,131.0,CHI,20572.0,1.0,IL,20575.0,39.0,2.0,1.0,20160428,O,O,M,1977.0,07262016,NaN,LX,59413424733.0,00008,WT
9,13213.0,2016.0,4.0,116.0,116.0,LOS,20545.0,1.0,CA,20553.0,35.0,2.0,1.0,20160401,O,O,M,1981.0,06292016,NaN,AA,55449792933.0,00109,WT
12,2711583.0,2016.0,4.0,148.0,112.0,FTL,20559.0,2.0,NaN,20565.0,54.0,2.0,1.0,20160415,G,O,M,1962.0,07132016,F,VES,56175860733.0,93724,WT
15,5960799.0,2016.0,4.0,245.0,245.0,SAI,20545.0,1.0,NaN,20550.0,30.0,2.0,1.0,20160615,P,D,M,1986.0,04132016,M,MU,44162582033.0,00763,CP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
974,3870400.0,2016.0,4.0,148.0,112.0,ATL,20565.0,1.0,CA,20571.0,35.0,2.0,1.0,20160421,O,O,M,1981.0,07192016,NaN,DL,56525407733.0,00131,WT
976,5018908.0,2016.0,4.0,111.0,111.0,MON,20571.0,1.0,FL,NaN,60.0,2.0,1.0,20160427,G,NaN,NaN,1956.0,07252016,M,RV,59339279133.0,01606,WT
977,4853913.0,2016.0,4.0,108.0,108.0,MIA,20570.0,2.0,NaN,20573.0,70.0,2.0,1.0,20160426,G,R,M,1946.0,07242016,F,VES,59277156733.0,91894,WT
989,1360834.0,2016.0,4.0,117.0,117.0,NYC,20552.0,1.0,NY,20556.0,38.0,2.0,1.0,20160408,O,O,M,1978.0,07062016,NaN,TK,55831485533.0,00001,WT


243 rows out of 1000 contain NaN after dropping unnecessary columns. Thus, can't be dropped.

In [70]:
# 13 rows out of 1000 contain "D/S" value for "dtaddto"
# needs to be dropped, then can change to date format
immigrationNew[immigrationNew["dtaddto"] == "D/S"]

,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,i94bir,i94visa,count,dtadfile,entdepa,entdepd,matflag,biryear,dtaddto,gender,airline,admnum,fltno,visatype
70,3599863.0,2016.0,4.0,582.0,582.0,HOU,20563.0,1.0,TX,20639.0,42.0,3.0,1.0,20160419,G,Q,M,1974.0,D/S,F,WN,93991159230.0,02831,F1
238,513953.0,2016.0,4.0,268.0,268.0,LOS,20547.0,1.0,CA,20627.0,19.0,3.0,1.0,20160403,G,O,M,1997.0,D/S,M,BR,92685611430.0,00012,F1
274,2209111.0,2016.0,4.0,268.0,268.0,SFR,20556.0,1.0,MA,20661.0,26.0,3.0,1.0,20160412,G,O,M,1990.0,D/S,F,UA,93329908030.0,00872,F1
337,5514177.0,2016.0,4.0,343.0,343.0,HOU,20573.0,1.0,TX,NaN,3.0,3.0,1.0,20160429,G,NaN,NaN,2013.0,D/S,M,EK,94889940930.0,00211,F2
415,1400880.0,2016.0,4.0,213.0,213.0,NEW,20552.0,1.0,NJ,NaN,2.0,3.0,1.0,20160408,T,NaN,NaN,2014.0,D/S,M,AI,725810385.0,191,F2
538,494733.0,2016.0,4.0,213.0,213.0,SFR,20547.0,1.0,CA,20589.0,32.0,3.0,1.0,20160403,T,O,M,1984.0,D/S,F,VS,676522085.0,41,F1
591,509225.0,2016.0,4.0,254.0,276.0,WAS,20547.0,1.0,VA,20603.0,17.0,3.0,1.0,20160403,T,O,M,1999.0,D/S,M,UA,675060085.0,804,F1
615,499578.0,2016.0,4.0,245.0,245.0,SFR,20547.0,1.0,CA,NaN,29.0,3.0,1.0,20160403,G,NaN,NaN,1987.0,D/S,F,UA,92641176030.0,00889,F1
621,5771544.0,2016.0,4.0,514.0,514.0,BAL,20574.0,1.0,MD,NaN,33.0,3.0,1.0,20160430,T,NaN,NaN,1983.0,D/S,M,WN,905856585.0,907,F1
684,4552209.0,2016.0,4.0,261.0,261.0,BLA,20568.0,3.0,NaN,20581.0,21.0,3.0,1.0,20160424,Z,K,M,1995.0,D/S,M,NaN,84429099430.0,00041,F1


In [71]:
# drop "dtaddto" equal to "D/S" 
# then can change "dtaddto" to date format since it contain "NaN"
immigrationPlus = immigrationNew[~(immigrationNew["dtaddto"] == "D/S")]

In [72]:
# change to date from SAS date numeric format
immigrationPlus["arrdate"] = pd.to_timedelta(immigrationPlus["arrdate"].dropna().astype(float), unit="D") + pd.Timestamp('1960-1-1')
immigrationPlus["depdate"] = pd.to_timedelta(immigrationPlus["depdate"].dropna().astype(float), unit="D") + pd.Timestamp('1960-1-1')

# change to date from other formats
immigrationPlus["dtadfile"] = pd.to_datetime(immigrationPlus["dtadfile"], format="%Y%m%d")
immigrationPlus["dtaddto"] = pd.to_datetime(immigrationPlus["dtaddto"], format="%m%d%Y")

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the

In [73]:
# change to int
immigrationPlus[["i94yr", "i94mon", "i94mode", 
                 "i94bir", "i94visa", "count", "biryear"]] = (immigrationPlus[["i94yr", "i94mon", "i94mode", "i94bir", 
                                                                               "i94visa", "count", "biryear"]].
                                                                                            astype(float).astype(int))

D:\Anaconda3\lib\site-packages\pandas\core\frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [74]:
# change to text
immigrationPlus[["cicid", "i94cit", "i94res", "admnum"]] = (immigrationPlus[["cicid", "i94cit", "i94res", "admnum"]].
                                                            apply(lambda x: x.astype(str).str.replace(".0", "", regex=False), axis=1))

In [75]:
# duplicates 
len(immigrationPlus) == len(immigrationPlus.drop_duplicates())

True

In [76]:
# data information
immigrationPlus.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 987 entries, 0 to 999
Data columns (total 24 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   cicid     987 non-null    object        
 1   i94yr     987 non-null    int32         
 2   i94mon    987 non-null    int32         
 3   i94cit    987 non-null    object        
 4   i94res    987 non-null    object        
 5   i94port   987 non-null    object        
 6   arrdate   987 non-null    datetime64[ns]
 7   i94mode   987 non-null    int32         
 8   i94addr   929 non-null    object        
 9   depdate   943 non-null    datetime64[ns]
 10  i94bir    987 non-null    int32         
 11  i94visa   987 non-null    int32         
 12  count     987 non-null    int32         
 13  dtadfile  987 non-null    datetime64[ns]
 14  entdepa   987 non-null    object        
 15  entdepd   946 non-null    object        
 16  matflag   946 non-null    object        
 17  biryear   987 no

In [77]:
# top 5
immigrationPlus.head()

,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,i94bir,i94visa,count,dtadfile,entdepa,entdepd,matflag,biryear,dtaddto,gender,airline,admnum,fltno,visatype
0,4084316,2016,4,209,209,HHW,2016-04-22,1,HI,2016-04-29,61,2,1,2016-04-22,G,O,M,1955,2016-07-20,F,JL,56582674633,00782,WT
1,4422636,2016,4,582,582,MCA,2016-04-23,1,TX,2016-04-24,26,2,1,2016-04-23,G,R,M,1990,2016-10-22,M,*GA,94361995930,XBLNG,B2
2,1195600,2016,4,148,112,OGG,2016-04-07,1,FL,2016-04-27,76,2,1,2016-04-07,G,O,M,1940,2016-07-05,M,LH,55780468433,00464,WT
3,5291768,2016,4,297,297,LOS,2016-04-28,1,CA,2016-05-07,25,2,1,2016-04-28,G,O,M,1991,2016-10-27,M,QR,94789696030,00739,B2
4,985523,2016,4,111,111,CHM,2016-04-06,3,NY,2016-04-09,19,2,1,2016-04-06,Z,K,M,1997,2016-07-04,F,NaN,42322572633,LAND,WT


##### temperature
1. Only include 2003 onwards of temperature for data usage
2. Only include US

In [78]:
# year range
# print(min(temperature.dt), max(temperature.dt)) ## 1743-11-01, 2013-09-01

In [79]:
# temperature in 2003 and onwards and in US
temperatureUS10Yr = temperature[(temperature.Country == "United States") 
            & (temperature.dt >= "2003-01-01")].drop("Country", axis=1)

In [80]:
# Uncertainty
#print(min(temperatureUS10Yr.AverageTemperatureUncertainty), max(temperatureUS10Yr.AverageTemperatureUncertainty))
## 0.049 1.3219999999999998

In [81]:
# NaN rows
temperatureUS10Yr[temperatureUS10Yr.isnull().any(axis=1)]

,dt,AverageTemperature,AverageTemperatureUncertainty,State
15106,2013-09-01,NaN,NaN,Alaska
154725,2013-09-01,NaN,NaN,Hawaii


In [82]:
# len(temperatureUS10Yr) ##6579

2 out of 6579 contains NaN. Thus. drop.

In [83]:
# drop NaN
temperatureUS10YrNoNa = temperatureUS10Yr.dropna() 

In [84]:
# change to date
temperatureUS10YrNoNa["dt"] = pd.to_datetime(temperatureUS10YrNoNa["dt"], format="%Y-%m-%d")

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [85]:
# to float and round to 1
temperatureUS10YrNoNa[["AverageTemperature", "AverageTemperatureUncertainty"]] = temperatureUS10YrNoNa[["AverageTemperature", "AverageTemperatureUncertainty"]].astype(float).round(1)

D:\Anaconda3\lib\site-packages\pandas\core\frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [86]:
# duplicates
len(temperatureUS10YrNoNa) == len(temperatureUS10YrNoNa.drop_duplicates())

True

In [87]:
# data information
temperatureUS10YrNoNa.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6577 entries, 10568 to 626439
Data columns (total 4 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   dt                             6577 non-null   datetime64[ns]
 1   AverageTemperature             6577 non-null   float64       
 2   AverageTemperatureUncertainty  6577 non-null   float64       
 3   State                          6577 non-null   object        
dtypes: datetime64[ns](1), float64(2), object(1)
memory usage: 256.9+ KB


In [88]:
# top 5
temperatureUS10YrNoNa.head()

,dt,AverageTemperature,AverageTemperatureUncertainty,State
10568,2003-01-01,4.8,0.1,Alabama
10569,2003-02-01,9.1,0.2,Alabama
10570,2003-03-01,14.6,0.2,Alabama
10571,2003-04-01,17.8,0.2,Alabama
10572,2003-05-01,22.6,0.2,Alabama


<a id="model"></a>

### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
* STAR Schema
* dimension tables
    * dim_airports ("State" PK; "num_airports")
    * dim_temperature ("State", PK; "min_tm"; "max_tm")
    * dim_population ("State", PK; age, gender, race population, household, etc)
* fact tables
    * fact_tourists ("cicid", PK; "State", FK)  <br>

#### 3.2 Mapping Out Data Pipelines

<!-- ![title](airport_data_dictionary.png) -->

##### Relationship between tables 

"State Code" in "cities", "region" in "airports", and "i94addr" in "immigration" are all related to US state codes, which can be a key to join all the three tables. And "State" in "temperature" and "State" in "cities" are all related to US state names, which can be a key to join the two tables, then adding "State Code" to "temperature".   
Use "State Code" to link all the tables together to create relevant dimension and fact tables.

##### State Codes

In [89]:
# airport region
np.array(sorted(airportUSNewNoNa.region.unique()))

array(['AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA',
       'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME',
       'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM',
       'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX',
       'U-A', 'UT', 'VA', 'VT', 'WA', 'WI', 'WV', 'WY'], dtype='<U3')

In [90]:
# city state
np.array(sorted(citiesNewNoDup["State Code"].unique()))

array(['AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA',
       'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME',
       'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM',
       'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX',
       'UT', 'VA', 'WA', 'WI'], dtype='<U2')

In [91]:
# compare state codes in cities and airports
set(np.array(sorted(citiesNewNoDup["State Code"].unique()))).difference(set(np.array(sorted(airportUSNewNoNa.region.unique()))))

set()

In [92]:
# compare state codes in cities and airports
set(np.array(sorted(airportUSNewNoNa.region.unique()))).difference(set(np.array(sorted(citiesNewNoDup["State Code"].unique()))))

{'U-A', 'VT', 'WV', 'WY'}

1. All the state codes in "cities" are in "airport".  
2. {'U-A', 'VT', 'WV', 'WY'} are in "airport" but not in "cities".

In [93]:
# immigration i94addr
np.array(sorted(immigrationNew["i94addr"].astype("str").unique()))

array(['AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'FL', 'GA', 'GU', 'HI',
       'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME', 'MI',
       'MN', 'MO', 'MP', 'MS', 'NC', 'NE', 'NH', 'NJ', 'NM', 'NV', 'NY',
       'OH', 'OK', 'OR', 'PA', 'PR', 'RI', 'SC', 'SW', 'TE', 'TN', 'TX',
       'UN', 'UT', 'VA', 'VQ', 'VT', 'WA', 'WI', 'nan'], dtype='<U3')

In [94]:
# compare state codes in cities and immigration
set(np.array(sorted(citiesNewNoDup["State Code"].unique()))).difference(set(np.array(sorted(immigrationNew["i94addr"].astype("str").unique()))))

{'AK', 'DE', 'MT', 'ND', 'SD'}

In [95]:
# compare state codes in cities and immigration
set(np.array(sorted(immigrationNew["i94addr"].astype("str").unique()))).difference(set(np.array(sorted(citiesNewNoDup["State Code"].unique()))))

{'GU', 'MP', 'PR', 'SW', 'TE', 'UN', 'VQ', 'VT', 'nan'}

1. {'AK', 'DE', 'MT', 'ND', 'SD'} are in "cities" but not in "immigration". 
2. {'GU', 'MP', 'PR', 'SW', 'TE', 'UN', 'VQ', 'VT'} are in "immigration" but not in "cities" (exclude "nan").

##### State Names

In [96]:
# state names in "cities" 
np.array(sorted(citiesNewNoDup.State.unique()))

array(['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California',
       'Colorado', 'Connecticut', 'Delaware', 'District of Columbia',
       'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana',
       'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland',
       'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi',
       'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire',
       'New Jersey', 'New Mexico', 'New York', 'North Carolina',
       'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania',
       'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee',
       'Texas', 'Utah', 'Virginia', 'Washington', 'Wisconsin'],
      dtype='<U20')

In [97]:
# state names in "temperature" 
np.array(sorted(temperatureUS10YrNoNa.State.unique()))

array(['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California',
       'Colorado', 'Connecticut', 'Delaware', 'District Of Columbia',
       'Florida', 'Georgia (State)', 'Hawaii', 'Idaho', 'Illinois',
       'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine',
       'Maryland', 'Massachusetts', 'Michigan', 'Minnesota',
       'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada',
       'New Hampshire', 'New Jersey', 'New Mexico', 'New York',
       'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon',
       'Pennsylvania', 'Rhode Island', 'South Carolina', 'South Dakota',
       'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington',
       'West Virginia', 'Wisconsin', 'Wyoming'], dtype='<U20')

In [98]:
# compare state names in "cities" and "temperature"
set(np.array(sorted(citiesNewNoDup.State.unique()))).difference(set(np.array(sorted(temperatureUS10YrNoNa.State.unique()))))

{'District of Columbia', 'Georgia'}

In [99]:
# compare state names in "cities" and "temperature"
set(np.array(sorted(temperatureUS10YrNoNa.State.unique()))).difference(set(np.array(sorted(citiesNewNoDup.State.unique()))))

{'District Of Columbia',
 'Georgia (State)',
 'Vermont',
 'West Virginia',
 'Wyoming'}

1. Change "District Of Columbia" to "District of Columbia" and "Georgia (State)" to "Georgia".
2. {'Vermont', 'West Virginia','Wyoming'} are in "temperature" not in "cities".

##### Staging tables

**SQLite Database**

In [100]:
# create a database called states
con = sqlite3.connect('states.db')

In [101]:
# create a cursor
cur = con.cursor()

In [102]:
# drop tables
cur.execute('''DROP TABLE IF EXISTS cities;''')
cur.execute('''DROP TABLE IF EXISTS airports;''')
cur.execute('''DROP TABLE IF EXISTS immigration;''')
cur.execute('''DROP TABLE IF EXISTS temperature;''')

In [103]:
# test
cur.execute('''SELECT name FROM sqlite_master WHERE type="table"''')
print(cur.fetchall())

[('sqlite_sequence',), ('dim_airports',), ('dim_cities',), ('dim_temperature',), ('fact_immigration',)]


##### airport

In [104]:
# top 5
# airportUSNewNoNa.head()

In [105]:
# data information
# airportUSNewNoNa.info()

In [106]:
# Create airport
cur.execute('''CREATE TABLE IF NOT EXISTS airports
               (airport_id INTEGER PRIMARY KEY AUTOINCREMENT,
               ident TEXT NOT NULL, 
               type TEXT NOT NULL, 
               name TEXT NOT NULL,
               elevation_ft REAL NOT NULL, 
               region TEXT NOT NULL,
               longitude REAL NOT NULL,
               latitude REAL NOT NULL);''')

In [107]:
# insert airportd from pandas dataframe
airportUSNewNoNa.to_sql("airports", con=con, if_exists="append", index=False)

##### Cities

In [108]:
# top 5
# citiesNewNoDup.head()

In [109]:
# data information
# citiesNewNoDup.info()

In [110]:
# Create cities
cur.execute('''CREATE TABLE IF NOT EXISTS cities 
               (city_id INTEGER PRIMARY KEY AUTOINCREMENT,
               City TEXT NOT NULL, 
               State TEXT NOT NULL, 
               "Median Age" REAL NOT NULL,
               "Male Population" INTEGER NOT NULL, 
               "Female Population" INTEGER NOT NULL,
               "Total Population" INTEGER NOT NULL,
               "Number of Veterans" INTEGER NOT NULL,
               "Foreign-born" INTEGER NOT NULL,
               "Average Household Size" REAL NOT NULL,
               "State Code" TEXT NOT NULL,
               "American Indian and Alaska Native" INTEGER,
               "Asian" INTEGER,
               "Black or African-American" INTEGER,
               "Hispanic or Latino" INTEGER NOT NULL,
               "White" INTEGER NOT NULL
               )''')

In [111]:
# insert cities from pandas dataframe
citiesNewNoDup.to_sql("cities", con=con, if_exists="append", index=False)

D:\Anaconda3\lib\site-packages\pandas\core\generic.py:2882: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  method=method,


##### Immigration

In [112]:
# top 5
# immigrationPlus.head()

In [113]:
# data information
# immigrationPlus.info()

In [114]:
# Create tourists
cur.execute('''CREATE TABLE IF NOT EXISTS immigration
               (cicid INTEGER PRIMARY KEY NOT NULL,
               i94yr INTEGER NOT NULL, 
               i94mon INTEGER NOT NULL, 
               i94cit TEXT NOT NULL,
               i94res TEXT NOT NULL,
               i94port TEXT NOT NULL,
               arrdate TEXT NOT NULL,
               i94mode INTEGER NOT NULL,
               i94addr TEXT,
               depdate TEXT,
               i94bir INTEGER NOT NULL,
               i94visa INTEGER NOT NULL,
               count INTEGER NOT NULL,
               dtadfile TEXT NOT NULL,
               visapost TEXT,
               entdepa TEXT NOT NULL,
               entdepd TEXT,
               matflag TEXT,
               biryear INTEGER NOT NULL,
               dtaddto TEXT NOT NULL,
               gender TEXT,
               airline TEXT,
               admnum TEXT NOT NULL,
               fltno TEXT,
               visatype TEXT NOT NULL)''')

In [115]:
# insert tourists from pandas dataframe
immigrationPlus.to_sql("immigration", con=con, if_exists="append", index=False)

##### Temperature

In [116]:
# top 5
# temperatureUS10YrNoNa.head()

In [117]:
# data information
# temperatureUS10YrNoNa.info()

In [118]:
# Create temperature
cur.execute('''CREATE TABLE IF NOT EXISTS temperature
               (temp_id INTEGER PRIMARY KEY AUTOINCREMENT,
               dt TEXT NOT NULL,
               AverageTemperature REAL NOT NULL,
               AverageTemperatureUncertainty REAL NOT NULL,
               State TEXT NOT NULL
               )''')

In [119]:
# insert tourists from pandas dataframe
temperatureUS10YrNoNa.to_sql("temperature", con=con, if_exists="append", index=False)

In [120]:
# Save (commit) the changes
con.commit()

# We can also close the connection if we are done with it.
# Just be sure any changes have been committed or they will be lost.
con.close()

##### Test

In [121]:
%sql sqlite:///states.db

In [122]:
%%sql
SELECT * FROM airports LIMIT 5;

 * sqlite:///states.db
Done.


airport_id,ident,type,name,elevation_ft,region,longitude,latitude
1,00AA,small_airport,Aero B Ranch Airport,3435.0,KS,-101.4739,38.704
2,00AK,small_airport,Lowell Field,450.0,AK,-151.696,59.9492
3,00AL,small_airport,Epps Airpark,820.0,AL,-86.7703,34.8648
4,00AS,small_airport,Fulton Airport,1100.0,OK,-97.818,34.9428
5,00AZ,small_airport,Cordes Airport,3810.0,AZ,-112.165,34.3056


In [123]:
%%sql
SELECT * FROM cities LIMIT 5;

 * sqlite:///states.db
Done.


city_id,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,American Indian and Alaska Native,Asian,Black or African-American,Hispanic or Latino,White
1,Wichita,Kansas,34.6,192354,197601,389955,23978,40270,2.56,KS,8791,25210,52592,65162,305910
2,Allen,Pennsylvania,33.5,60626,59581,120207,5691,19652,2.67,PA,538113.5,133507895,1115206570,2958805307.5,3709384920
3,Danbury,Connecticut,37.3,43435,41227,84662,3752,25675,2.74,CT,1086,7350,8454,25145,55917
4,Nashville,Tennessee,34.1,314231,340365,654596,27942,88193,2.39,TN,5474,27355,188844,67526,432447
5,Stamford,Connecticut,35.4,64941,63936,128877,2269,44003,2.7,CT,1416,11013,24329,33197,85620


In [124]:
%%sql
SELECT * FROM immigration LIMIT 5;

 * sqlite:///states.db
Done.


cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,i94bir,i94visa,count,dtadfile,visapost,entdepa,entdepd,matflag,biryear,dtaddto,gender,airline,admnum,fltno,visatype
13208,2016,4,116,116,LOS,2016-04-01 00:00:00,1,CA,2016-04-30 00:00:00,29,2,1,2016-04-01 00:00:00,None,G,O,M,1987,2016-06-29 00:00:00,M,VS,55442244033,00007,WT
13213,2016,4,116,116,LOS,2016-04-01 00:00:00,1,CA,2016-04-09 00:00:00,35,2,1,2016-04-01 00:00:00,None,O,O,M,1981,2016-06-29 00:00:00,None,AA,55449792933,00109,WT
13826,2016,4,117,117,ATL,2016-04-01 00:00:00,1,SC,2016-04-09 00:00:00,44,1,1,2016-04-01 00:00:00,None,G,O,M,1972,2016-06-29 00:00:00,M,AF,55459078733,00688,WB
17786,2016,4,123,123,NYC,2016-04-01 00:00:00,1,NE,2016-04-12 00:00:00,31,1,1,2016-04-01 00:00:00,None,O,O,M,1985,2016-06-29 00:00:00,None,VS,55455177333,00009,WB
18310,2016,4,123,123,SEA,2016-04-01 00:00:00,1,CA,2016-04-04 00:00:00,45,2,1,2016-04-01 00:00:00,None,G,I,M,1971,2016-06-29 00:00:00,M,DL,55421541133,00143,WT


In [125]:
%%sql
SELECT * FROM temperature LIMIT 5;

 * sqlite:///states.db
Done.


temp_id,dt,AverageTemperature,AverageTemperatureUncertainty,State
1,2003-01-01 00:00:00,4.8,0.1,Alabama
2,2003-02-01 00:00:00,9.1,0.2,Alabama
3,2003-03-01 00:00:00,14.6,0.2,Alabama
4,2003-04-01 00:00:00,17.8,0.2,Alabama
5,2003-05-01 00:00:00,22.6,0.2,Alabama


##### Normalization

dim_airports

In [141]:
# %%sql
# SELECT DISTINCT tb.region,  totalAirport FROM
#     (SELECT region, COUNT(ident) AS totalAirport
#     FROM airports
#     GROUP BY 1) tb
#     WHERE region IN (SELECT DISTINCT [State Code] FROM cities)
#     AND region IN (SELECT DISTINCT i94addr FROM immigration)
#     -- ORDER BY 1, 2 DESC

In [127]:
# create a database called states
con = sqlite3.connect('states.db')

# create a cursor
cur = con.cursor()

# drop tables
cur.execute('''DROP TABLE IF EXISTS dim_airports;''')
cur.execute('''CREATE TABLE IF NOT EXISTS dim_airports AS
                SELECT DISTINCT tb.region,  totalAirport FROM
                    (SELECT region, COUNT(ident) AS totalAirport
                    FROM airports
                    GROUP BY 1) tb
                    WHERE region IN (SELECT DISTINCT [State Code] FROM cities)
                    AND region IN (SELECT DISTINCT i94addr FROM immigration)
;''')

# Save (commit) the changes
con.commit()

# We can also close the connection if we are done with it.
# Just be sure any changes have been committed or they will be lost.
con.close()

In [128]:
%%sql
SELECT * FROM dim_airports LIMIT 5

 * sqlite:///states.db
Done.


region,totalAirport
AL,197
AR,274
AZ,214
CA,547
CO,289


dim_cities

In [140]:
# %%sql
# SELECT DISTINCT tb.* FROM
#     (SELECT [State Code], 
#     ROUND(AVG([Median Age]), 1) AS [Median Age],
#     SUM([Male Population]) AS [Male Population],
#     SUM([Female Population]) AS [Female Population],
#     SUM([Total Population]) AS [Total Population],
#     SUM([Number of Veterans]) AS [Number of Veterans],
#     SUM([Foreign-born]) AS [Foreign-born],
#     ROUND(AVG([Average Household Size]), 1) AS [Average Household Size],
#     ROUND(SUM([American Indian and Alaska Native]), 1) AS [American Indian and Alaska Native],
#     ROUND(SUM([Asian]), 1) AS [Asian],
#     ROUND(SUM([Black or African-American]), 1) AS [Black or African-American],
#     ROUND(SUM([Hispanic or Latino]), 1) AS [Hispanic or Latino],
#     ROUND(SUM([White]), 1) AS [White] FROM cities
#     GROUP BY [State Code]) tb
#     WHERE [State Code] IN (SELECT DISTINCT region FROM airports)
#     AND [State Code] IN (SELECT DISTINCT i94addr FROM immigration)
#     -- JOIN airports 
#     -- ON tb.[State Code] = airports.[region]
#     -- JOIN  tourists
#     -- ON tb.[State Code] = tourists.[i94addr]        
#     -- ORDER BY 1

In [130]:
# create a database called states
con = sqlite3.connect('states.db')
# create a cursor
cur = con.cursor()
# drop tables
cur.execute('''DROP TABLE IF EXISTS dim_cities;''')
cur.execute(''' CREATE TABLE IF NOT EXISTS dim_cities AS
                SELECT DISTINCT tb.* FROM
                    (SELECT [State Code], 
                    ROUND(AVG([Median Age]), 1) AS [Median Age],
                    SUM([Male Population]) AS [Male Population],
                    SUM([Female Population]) AS [Female Population],
                    SUM([Total Population]) AS [Total Population],
                    SUM([Number of Veterans]) AS [Number of Veterans],
                    SUM([Foreign-born]) AS [Foreign-born],
                    ROUND(AVG([Average Household Size]), 1) AS [Average Household Size],
                    ROUND(SUM([American Indian and Alaska Native]), 1) AS [American Indian and Alaska Native],
                    ROUND(SUM([Asian]), 1) AS [Asian],
                    ROUND(SUM([Black or African-American]), 1) AS [Black or African-American],
                    ROUND(SUM([Hispanic or Latino]), 1) AS [Hispanic or Latino],
                    ROUND(SUM([White]), 1) AS [White] FROM cities
                    GROUP BY [State Code]) tb
                    WHERE [State Code] IN (SELECT DISTINCT region FROM airports)
                    AND [State Code] IN (SELECT DISTINCT i94addr FROM immigration)
;''')
# Save (commit) the changes
con.commit()

# We can also close the connection if we are done with it.
# Just be sure any changes have been committed or they will be lost.
con.close()

In [131]:
%%sql
SELECT * FROM dim_cities LIMIT 5

 * sqlite:///states.db
Done.


State Code,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,American Indian and Alaska Native,Asian,Black or African-American,Hispanic or Latino,White
AL,36.2,497248,552381,1049629,71543,52154,2.4,8084.0,28769.0,521068.0,39313.0,498920.0
AR,32.8,286479,303400,589879,31704,62108,2.5,33023352.0,23556829.5,453271144.5,125475045.5,34415366940.5
AZ,35.0,2227455,2272087,4499542,264505,682313,2.8,9211122.0,795636836.5,412898838.0,4530669285.0,80923977646.5
CA,36.2,12278281,12544179,24822460,928270,7448257,3.1,45771591.5,2331666489.5,6229933287.5,15078379078.5,151563347595.5
CO,35.8,1454619,1481050,2935669,187896,337631,2.6,48537905.0,2034199483.5,1211720864.5,45744218409.5,83733626186.5


dim_temperature

There is no funcion like PIVOT in Sqlite. Thus. use the following SQL query to achieve the function.

In [139]:
# %%sql 
# WITH season AS (
# SELECT 
#     CASE 
#     WHEN strftime('%m', dt) IN ('03', '04', '05') THEN 'Spring'
#     WHEN strftime('%m', dt) IN ('06', '07', '08') THEN 'Summer'
#     WHEN strftime('%m', dt) IN ('09', '10', '11') THEN 'Fall'
#     WHEN strftime('%m', dt) IN ('12', '01', '02') THEN 'Winter'
#     END Season, 
#     CASE 
#     WHEN State = 'Georgia (State)' THEN 'Georgia'
#     ELSE State
#     END AS State
#     , 
#     ROUND(AVG(AverageTemperature), 1) AS AverageTemperature 
#     FROM temperature
#     GROUP BY 1, 2
# )
# SELECT DISTINCT cities.[State Code], FallAvgTemp, SummerAvgTemp, SpringAvgTemp, WinterAvgTemp FROM
#     (SELECT State, 
#     CASE WHEN Season = 'Fall' THEN AverageTemperature END AS FallAvgTemp FROM season) fall 
#     JOIN (
#     SELECT State, 
#     CASE WHEN Season = 'Summer' THEN AverageTemperature END AS SummerAvgTemp FROM season 
#     ) summer
#     ON fall.State = Summer.State
#     JOIN (
#     SELECT State, 
#     CASE WHEN Season = 'Spring' THEN AverageTemperature END AS SpringAvgTemp FROM season     
#     ) spring
#     ON fall.State = spring.State
#     JOIN (
#     SELECT State, 
#     CASE WHEN Season = 'Winter' THEN AverageTemperature END AS WinterAvgTemp FROM season        
#     ) winter
#     ON fall.State = winter.State
#     JOIN cities 
#     ON fall.State = cities.State
#     WHERE cities.[State Code] IN (SELECT DISTINCT region FROM airports)
#     AND cities.[State Code] IN (SELECT DISTINCT i94addr FROM immigration)
#     AND FallAvgTemp IS NOT NULL AND SummerAvgTemp IS NOT NULL AND SpringAvgTemp IS NOT NULL AND WinterAvgTemp IS NOT NULL
#     -- ORDER BY 1

In [133]:
# create a database called states
con = sqlite3.connect('states.db')
# create a cursor
cur = con.cursor()
# drop tables
cur.execute('''DROP TABLE IF EXISTS dim_temperature;''')
cur.execute(''' CREATE TABLE IF NOT EXISTS dim_temperature AS
                WITH season AS (
                SELECT 
                    CASE 
                    WHEN strftime('%m', dt) IN ('03', '04', '05') THEN 'Spring'
                    WHEN strftime('%m', dt) IN ('06', '07', '08') THEN 'Summer'
                    WHEN strftime('%m', dt) IN ('09', '10', '11') THEN 'Fall'
                    WHEN strftime('%m', dt) IN ('12', '01', '02') THEN 'Winter'
                    END Season, 
                    CASE 
                    WHEN State = 'Georgia (State)' THEN 'Georgia'
                    WHEN State = 'District Of Columbia' THEN 'District of Columbia'
                    ELSE State
                    END AS State
                    , 
                    ROUND(AVG(AverageTemperature), 1) AS AverageTemperature 
                    FROM temperature
                    GROUP BY 1, 2
                )
                SELECT DISTINCT cities.[State Code], FallAvgTemp, SummerAvgTemp, SpringAvgTemp, WinterAvgTemp FROM
                    (SELECT State, 
                    CASE WHEN Season = 'Fall' THEN AverageTemperature END AS FallAvgTemp FROM season) fall 
                    JOIN (
                    SELECT State, 
                    CASE WHEN Season = 'Summer' THEN AverageTemperature END AS SummerAvgTemp FROM season 
                    ) summer
                    ON fall.State = Summer.State
                    JOIN (
                    SELECT State, 
                    CASE WHEN Season = 'Spring' THEN AverageTemperature END AS SpringAvgTemp FROM season     
                    ) spring
                    ON fall.State = spring.State
                    JOIN (
                    SELECT State, 
                    CASE WHEN Season = 'Winter' THEN AverageTemperature END AS WinterAvgTemp FROM season        
                    ) winter
                    ON fall.State = winter.State
                    JOIN cities 
                    ON fall.State = cities.State
                    WHERE cities.[State Code] IN (SELECT DISTINCT region FROM airports)
                    AND cities.[State Code] IN (SELECT DISTINCT i94addr FROM immigration)
                    AND FallAvgTemp IS NOT NULL AND SummerAvgTemp IS NOT NULL AND SpringAvgTemp IS NOT NULL AND WinterAvgTemp IS NOT NULL
;''')
# Save (commit) the changes
con.commit()

# We can also close the connection if we are done with it.
# Just be sure any changes have been committed or they will be lost.
con.close()

In [134]:
%%sql
SELECT * FROM dim_temperature LIMIT 5

 * sqlite:///states.db
Done.


State Code,FallAvgTemp,SummerAvgTemp,SpringAvgTemp,WinterAvgTemp
AL,18.6,26.9,17.9,8.4
AZ,17.1,26.5,15.3,6.1
AR,17.3,26.8,16.6,5.6
CA,16.1,23.4,13.5,7.2
CO,8.6,19.2,7.0,-3.4


fact_immigration

In [138]:
# %%sql
# SELECT * FROM 
# immigration  
# WHERE i94addr IN
# (SELECT DISTINCT [State Code] FROM cities)
# AND i94addr IN (SELECT DISTINCT region FROM airports)

In [136]:
# create a database called states
con = sqlite3.connect('states.db')
# create a cursor
cur = con.cursor()
# drop tables
cur.execute('''DROP TABLE IF EXISTS fact_immigration;''')
cur.execute(''' CREATE TABLE IF NOT EXISTS fact_immigration AS
                SELECT * FROM 
                immigration  
                WHERE i94addr IN
                (SELECT DISTINCT [State Code] FROM cities)
                AND i94addr IN (SELECT DISTINCT region FROM airports)
;''')
# Save (commit) the changes
con.commit()

# We can also close the connection if we are done with it.
# Just be sure any changes have been committed or they will be lost.
con.close()

In [137]:
%%sql
SELECT * FROM fact_immigration LIMIT 5

 * sqlite:///states.db
Done.


cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,i94bir,i94visa,count,dtadfile,visapost,entdepa,entdepd,matflag,biryear,dtaddto,gender,airline,admnum,fltno,visatype
13208,2016,4,116,116,LOS,2016-04-01 00:00:00,1,CA,2016-04-30 00:00:00,29,2,1,2016-04-01 00:00:00,None,G,O,M,1987,2016-06-29 00:00:00,M,VS,55442244033,00007,WT
13213,2016,4,116,116,LOS,2016-04-01 00:00:00,1,CA,2016-04-09 00:00:00,35,2,1,2016-04-01 00:00:00,None,O,O,M,1981,2016-06-29 00:00:00,None,AA,55449792933,00109,WT
13826,2016,4,117,117,ATL,2016-04-01 00:00:00,1,SC,2016-04-09 00:00:00,44,1,1,2016-04-01 00:00:00,None,G,O,M,1972,2016-06-29 00:00:00,M,AF,55459078733,00688,WB
17786,2016,4,123,123,NYC,2016-04-01 00:00:00,1,NE,2016-04-12 00:00:00,31,1,1,2016-04-01 00:00:00,None,O,O,M,1985,2016-06-29 00:00:00,None,VS,55455177333,00009,WB
18310,2016,4,123,123,SEA,2016-04-01 00:00:00,1,CA,2016-04-04 00:00:00,45,2,1,2016-04-01 00:00:00,None,G,I,M,1971,2016-06-29 00:00:00,M,DL,55421541133,00143,WT


<a id='check'></a>

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

##### Check the relationship among the normalized tables

In [143]:
%%sql
SELECT DISTINCT i94addr, dc.[State Code], da.region, dt.[State Code] FROM fact_immigration f
LEFT JOIN dim_cities dc
ON f.i94addr = dc.[State Code]
LEFT JOIN dim_airports da
ON f.i94addr = da.region
LEFT JOIN dim_temperature dt
ON f.i94addr = dt.[State Code]
ORDER BY 1

 * sqlite:///states.db
Done.


i94addr,State Code,region,State Code_1
AL,AL,AL,AL
AR,AR,AR,AR
AZ,AZ,AZ,AZ
CA,CA,CA,CA
CO,CO,CO,CO
CT,CT,CT,CT
DC,DC,DC,DC
FL,FL,FL,FL
GA,GA,GA,GA
HI,HI,HI,HI


In [144]:
%%sql
SELECT COUNT(*), MIN([State Code]), MAX([State Code]) FROM dim_cities

 * sqlite:///states.db
Done.


COUNT(*),MIN(,MAX(
43,AL,WI


In [385]:
%%sql
SELECT COUNT(*), MIN([State Code]), MAX([State Code]) FROM dim_temperature

 * sqlite:///states.db
Done.


COUNT(*),MIN(,MAX(
43,AL,WI


In [386]:
%%sql
SELECT COUNT(DISTINCT region), MIN(region), MAX(region) FROM dim_airports

 * sqlite:///states.db
Done.


COUNT(DISTINCT region),MIN(region),MAX(region)
43,AL,WI


In [387]:
%%sql
SELECT COUNT(DISTINCT i94addr), MIN(i94addr), MAX(i94addr) FROM fact_tourists

 * sqlite:///states.db
Done.


COUNT(DISTINCT i94addr),MIN(i94addr),MAX(i94addr)
43,AL,WI


All the state codes in the dimension tables and fact table can be matched. 

##### Check the normalized tables with the original tables

**cities**

In [150]:
%%sql
SELECT * FROM dim_cities
WHERE [State Code] = 'AL'

 * sqlite:///states.db
Done.


State Code,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,American Indian and Alaska Native,Asian,Black or African-American,Hispanic or Latino,White
AL,36.2,497248,552381,1049629,71543,52154,2.4,8084.0,28769.0,521068.0,39313.0,498920.0


In [152]:
# original cities
cities[cities["State Code"] == "AL"]

,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,Race,Count
88,Tuscaloosa,Alabama,29.1,47293,51045,98338,3647,4706,2.67,AL,White,52603
124,Dothan,Alabama,38.9,32172,35364,67536,6334,1699,2.59,AL,Asian,1175
138,Hoover,Alabama,38.5,38040,46799,84839,4819,8229,2.58,AL,Black or African-American,18191
140,Dothan,Alabama,38.9,32172,35364,67536,6334,1699,2.59,AL,Black or African-American,23243
224,Tuscaloosa,Alabama,29.1,47293,51045,98338,3647,4706,2.67,AL,Asian,2733
400,Birmingham,Alabama,35.6,102122,112789,214911,13212,8258,2.21,AL,Black or African-American,157985
415,Birmingham,Alabama,35.6,102122,112789,214911,13212,8258,2.21,AL,Asian,1500
507,Huntsville,Alabama,38.1,91764,97350,189114,16637,12691,2.18,AL,Black or African-American,61561
568,Birmingham,Alabama,35.6,102122,112789,214911,13212,8258,2.21,AL,White,51728
595,Huntsville,Alabama,38.1,91764,97350,189114,16637,12691,2.18,AL,Hispanic or Latino,10887


The information listed in the "dim_cities" is the same as the information listed in the original "cities".

**airports**

In [153]:
%%sql
SELECT * FROM dim_airports 
WHERE region = 'AL'

 * sqlite:///states.db
Done.


region,totalAirport
AL,197


In [163]:
# original airport
(airport[(airport.iso_region == 'US-AL') &
        ~(airport.type == "closed") & 
        airport.type.str.contains("airport") & 
        ~airport.elevation_ft.isna()].
        drop(["continent","iata_code", "gps_code", "local_code", "municipality"], axis=1).drop_duplicates())

,ident,type,name,elevation_ft,iso_country,iso_region,coordinates
3,00AL,small_airport,Epps Airpark,820,US,US-AL,"-86.77030181884766, 34.86479949951172"
65,01AL,small_airport,Ware Island Airport,344,US,US-AL,"-86.51390075683594, 32.94599914550781"
129,02AL,small_airport,Bass Field,61,US,US-AL,"-87.76439666748047, 30.37150001525879"
605,0AL1,small_airport,Resort Airport,97,US,US-AL,"-87.65689849853516, 30.443899154663086"
609,0AL5,small_airport,Flomaton Airport,247,US,US-AL,"-87.25279998779297, 31.03219985961914"
...,...,...,...,...,...,...,...
49078,US-0064,small_airport,Hickory Grove STOLport,700,US,US-AL,"-85.702796936, 33.901699066199996"
49623,US-0608,small_airport,Hazel Green Acres Airport,908,US,US-AL,"-86.692304, 34.980471"
49697,US-0682,small_airport,J5 Mike Airport,680,US,US-AL,"-86.222286, 34.666347"
49700,US-0685,small_airport,Rocky Springs Airpark,649,US,US-AL,"-87.077074, 33.988596"


Number of airports in the "dim_airports" can be matched to the number in the original "airport" table.

**temperature**

In [164]:
%%sql
SELECT * FROM dim_temperature
WHERE [State Code] = 'AL'

 * sqlite:///states.db
Done.


State Code,FallAvgTemp,SummerAvgTemp,SpringAvgTemp,WinterAvgTemp
AL,18.6,26.9,17.9,8.4


In [176]:
# original temperature table
spring = round(temperature[(temperature.dt.str.contains("-03-|-04-|-05-")) & (temperature.State == "Alabama")]["AverageTemperature"].astype(float).mean(), 1)
summer = round(temperature[(temperature.dt.str.contains("-06-|-07-|-08-")) & (temperature.State == "Alabama")]["AverageTemperature"].astype(float).mean(), 1)
autumn = round(temperature[(temperature.dt.str.contains("-09-|-10-|-11-")) & (temperature.State == "Alabama")]["AverageTemperature"].astype(float).mean(), 1)
winter = round(temperature[(temperature.dt.str.contains("-12-|-01-|-02-")) & (temperature.State == "Alabama")]["AverageTemperature"].astype(float).mean(), 1)
print("Fall:{}\tSummer:{}\tSpring:{}\tWinter:{}".format(autumn, summer, spring, winter))

Fall:17.4	Summer:26.0	Spring:17.0	Winter:8.0


The temperature in "dim_temperature" is a little higher than original "temperature" table.

**immigration**

In [177]:
%%sql
SELECT * FROM fact_immigration
WHERE i94addr = "AL"

 * sqlite:///states.db
Done.


cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,i94bir,i94visa,count,dtadfile,visapost,entdepa,entdepd,matflag,biryear,dtaddto,gender,airline,admnum,fltno,visatype
1193133,2016,4,148,112,CHI,2016-04-07 00:00:00,1,AL,2016-04-17 00:00:00,58,2,1,2016-04-07 00:00:00,None,O,O,M,1958,2016-07-05 00:00:00,None,AF,55763314933,00682,WT
2487808,2016,4,111,111,MIA,2016-04-14 00:00:00,1,AL,2016-05-02 00:00:00,67,2,1,2016-04-14 00:00:00,None,G,O,M,1949,2016-07-12 00:00:00,F,AF,56156579533,00090,WT
4756602,2016,4,520,520,MIA,2016-04-25 00:00:00,1,AL,2016-06-21 00:00:00,5,2,1,2016-04-25 00:00:00,None,G,O,M,2011,2016-10-24 00:00:00,F,AA,94500896530,00982,B2
4882428,2016,4,148,112,ATL,2016-04-26 00:00:00,1,AL,2016-05-01 00:00:00,53,1,1,2016-04-26 00:00:00,None,G,O,M,1963,2016-07-24 00:00:00,M,DL,59296675233,00015,WB
5081528,2016,4,254,276,SFR,2016-04-27 00:00:00,1,AL,2016-04-29 00:00:00,48,1,1,2016-04-27 00:00:00,None,O,O,M,1968,2016-07-25 00:00:00,None,EV,59327444033,05510,WB


In [178]:
# original immigration
immigration[immigration.i94addr == "AL"]

,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,i94bir,i94visa,count,dtadfile,visapost,occup,entdepa,entdepd,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
326,2487808.0,2016.0,4.0,111.0,111.0,MIA,20558.0,1.0,AL,20576.0,67.0,2.0,1.0,20160414,NaN,NaN,G,O,NaN,M,1949.0,07122016,F,NaN,AF,56156579533.0,00090,WT
576,4756602.0,2016.0,4.0,520.0,520.0,MIA,20569.0,1.0,AL,20626.0,5.0,2.0,1.0,20160425,BGN,NaN,G,O,NaN,M,2011.0,10242016,F,NaN,AA,94500896530.0,00982,B2
799,5081528.0,2016.0,4.0,254.0,276.0,SFR,20571.0,1.0,AL,20573.0,48.0,1.0,1.0,20160427,NaN,NaN,O,O,NaN,M,1968.0,07252016,NaN,NaN,EV,59327444033.0,05510,WB
816,1193133.0,2016.0,4.0,148.0,112.0,CHI,20551.0,1.0,AL,20561.0,58.0,2.0,1.0,20160407,NaN,NaN,O,O,NaN,M,1958.0,07052016,NaN,NaN,AF,55763314933.0,00682,WT
908,4882428.0,2016.0,4.0,148.0,112.0,ATL,20570.0,1.0,AL,20575.0,53.0,1.0,1.0,20160426,NaN,NaN,G,O,NaN,M,1963.0,07242016,M,NaN,DL,59296675233.0,00015,WB


The information listed in the "fact_immigration" is the same as the information in the original "immigration" table.

In conclusion, the information in the original tables is pretty close to the information in the normalized tables.

#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

In [392]:
# %%sql
# SELECT * FROM dim_cities LIMIT 5

* **dim_cities**:
    * "State Code" (PK): Code for states.
    * "Median Age": Median Age for population for US 2015 Census.
    * "Male Population": Male population for US 2015 Census.
    * "Female Population": Female Population for US 2015 Census.
    * "Total Population": Total Population for US 2015 Census.
    * "Number of Veterans": Number of Veterans for US 2015 Census.
    * "Foreign-born": Foreign-born population for US 2015 Census.
    * "Average Household Size": Average Household Size for US 2015 Census.
    * "American Indian and Alaska Native": American Indian and Alaska Native population for US 2015 Census.
    * "Asian": Asian population for US 2015 Census.
    * "Hispanic or Latino": Hispanic or Latino population for US 2015 Census.
    * "White": White population for US 2015 Census.

In [393]:
# %%sql
# SELECT * FROM dim_airports LIMIT 5

* **dim_airports**:
    * "region" (PK): Code for states.
    * "totalAirport": Number of airports in the state (including small, medium, and large airports).

In [394]:
# %%sql
# SELECT * FROM dim_temperature LIMIT 5

* **dim_temperature**:
    * "State Code" (PK): Code for states.
    * "FallAvgTemp": Average temperature in Fall (Sep, Oct, Nov) from 2003 onwards.
    * "SummerAvgTemp": Average temperature in Summer (Jun, Jul, Oct) from 2003 onwards.
    * "	SpringAvgTemp": Average temperature in Spring (Mar, Apr, May) from 2003 onwards.
    * "WinterAvgTemp": Average temperature in Winter (Dec, Jan, Feb) from 2003 onwards.

In [179]:
# %%sql
# SELECT * FROM fact_tourists LIMIT 5;

* **fact_tourists**:
    * "cicid" (PK): cic id for tourists.
    * "i94yr": i94 filling year.
    * "i94mon": i94 filling month.
    * "i94cit": country codes.
    * "i94res": country codes.
    * "i94port": port initials.
    * "arrdate": Arrival Date in the USA.
    * "i94mode": 1 = 'Air, '2 = 'Sea', 3 = 'Land', 9 = 'Not reported' ;.
    * "i94addr" (FK): US state codes.
    * "depdate": Departure Date from the USA.
    * "i94bir": Respondant birth year.
    * "i94visa": visa codes, 1 = Business, 2 = Pleasure, 3 = Student.
    * "count": Used for summary statistics.
    * "dtadfile": Date added to I-94 Files.
    * "visapost":  Department of State where where Visa was issued.
    * "entdepa": Arrival Flag - admitted or paroled into the U.S..
    * "entdepd": Departure Flag - Departed, lost I-94 or is deceased.
    * "matflag": Match flag - Match of arrival and departure records.    
    * "biryear": 4 digit year of birth.
    * "dtaddto": Date to which admitted to U.S. (allowed to stay until).
    * "gender": Non-immigrant sex.
    * "airline": Airline used to arrive in U.S..
    * "admnum": Admission Number.
    * "fltno": Flight number of Airline used to arrive in U.S..
    * "visatype": Class of admission legally admitting the non-immigrant to temporarily stay in U.S..       

#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.